In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time

import os

In [2]:
file = "diy_mco_28-April'25.csv"
raw_data=pd.read_csv(file,low_memory=False,memory_map=True)
data=pd.read_csv("format.csv",memory_map=True)
table_data=pd.read_csv("table_data.csv",memory_map=True)



In [3]:
raw_data.head(3)

,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp
0,60615636,ebbee93a-4e0b-4e72-a337-31bba8873c8f,60615636,156,LEAD_SUCCESSFULLY_CLOSED,2025-04-15 21:46:45.000,2025-04-16 09:29:36.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,gJkJJy07871731905616,2025-04-16 09:25:39.000,NaN,2025-04-15 21:53:55.000,2025-04-16 09:25:33.000,2025-04-16 09:25:33.000
1,1407252309,a4290aea-f485-45fd-9bd6-350fd37b324a,1407252309,2136,PARTNER_PROFILE_EDIT_FAILURE,2025-04-15 06:50:48.000,2025-04-17 22:16:55.000,ybl_rekyc_update,NaN,INDIVIDUAL,NaN,NaN,True,JyIILu76408891542729,2022-01-26 19:56:33.000,NaN,NaN,NaN,2025-04-17 22:16:55.000
2,1439811339,223ebbf0-85ee-45f2-acc1-d77324831c09,1439811339,156,LEAD_SUCCESSFULLY_CLOSED,2025-04-16 12:44:47.000,2025-04-16 13:15:34.000,NaN,NaN,INDIVIDUAL,NaN,"{""QC_REJECTED"":1,""PANEL_SUCCESS"":1}",True,AxdVGE78268205132974,2025-04-16 13:07:09.000,NaN,2025-04-16 13:04:25.000,2025-04-16 13:06:56.000,2025-04-16 13:06:56.000


In [4]:

raw_data.insert(5,'q',0)

In [5]:
table_map = table_data.set_index('NODE ID')['Lead Status'].to_dict()
raw_data['q'] = raw_data['workflow_node_id'].map(table_map)

raw_data['q'].unique()

array(['Merchant Live', nan, 'Documents Rejected (Pending at Sales Team)',
       'Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections) ',
       'Lead Created/Posted (Pending at Sales team for  Document Upload)'],
      dtype=object)

In [6]:
raw_data=raw_data.dropna(subset=['q'])

raw_data['q'].value_counts()

Merchant Live                                                       115332
Documents Rejected (Pending at Sales Team)                           16014
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)          741
Lead Created/Posted (Pending at Sales team for  Document Upload)       109
Name: q, dtype: int64

In [7]:
raw_data['remove_date']=raw_data['updated_at'].copy()

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])

raw_data['remove_date'] = raw_data['remove_date'].dt.strftime('%Y-%m-%d')

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])


In [8]:

current_timestamp = pd.Timestamp.now().normalize()

raw_data=raw_data[raw_data['remove_date']!=current_timestamp]

In [9]:
raw_data=raw_data.drop(columns=['remove_date'])
len(raw_data)


130707

In [10]:
raw_data['solution_type_level_2'].unique()

array([nan, 'display_name_update', 'ybl_rekyc_update',
       'business_name_update', 'diy_upgrade', 'auth_signatory_update',
       'pan_addition', 'billing_address_update',
       'registered_address_update', 'gst_update', 'diy_edc'], dtype=object)

In [11]:
raw_data=raw_data[raw_data['solution_type_level_2'].isin(['diy_upgrade','']) | raw_data['solution_type_level_2'].isna()]
len(raw_data)

43905

In [12]:
raw_data['solution_type_level_2'].unique()

array([nan, 'diy_upgrade'], dtype=object)

In [13]:
print(len(raw_data))
raw_data.head(3)

43905


,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,q,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp
0,60615636,ebbee93a-4e0b-4e72-a337-31bba8873c8f,60615636,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-15 21:46:45.000,2025-04-16 09:29:36.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,gJkJJy07871731905616,2025-04-16 09:25:39.000,NaN,2025-04-15 21:53:55.000,2025-04-16 09:25:33.000,2025-04-16 09:25:33.000
2,1439811339,223ebbf0-85ee-45f2-acc1-d77324831c09,1439811339,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-16 12:44:47.000,2025-04-16 13:15:34.000,NaN,NaN,INDIVIDUAL,NaN,"{""QC_REJECTED"":1,""PANEL_SUCCESS"":1}",True,AxdVGE78268205132974,2025-04-16 13:07:09.000,NaN,2025-04-16 13:04:25.000,2025-04-16 13:06:56.000,2025-04-16 13:06:56.000
6,1312985574,ae574c05-c9ad-42f5-a645-1d674fe4a7a7,1312985574,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-11 20:52:40.000,2025-04-12 08:55:59.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,isceDl72329004783334,2025-04-12 08:48:19.000,NaN,2025-04-11 21:03:16.000,2025-04-12 08:48:14.000,2025-04-12 08:48:14.000


In [14]:
raw_data.columns

Index(['cust_id', 'lead_id', 'fse_cust_id', 'workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at', 'solution_type_level_2',
       'solution_type_level_3', 'entity_type', 'products_context',
       'sbstage_count', 'name_match_status', 'mid', 'mid_creation_date',
       'agent_team', 'qc_pending_max_created', 'panelqc_max_created',
       'final_timestamp'],
      dtype='object')

In [15]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [16]:
data[['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp',
       'Product_Context', 'Name_Match_Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp',
       'Stage movement count', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3','solution_type']]=raw_data[['cust_id', 'lead_id','entity_type','workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at','products_context','name_match_status','final_timestamp','panelqc_max_created','qc_pending_max_created','sbstage_count','fse_cust_id','agent_team','mid','mid_creation_date','solution_type_level_2','solution_type_level_3']]
       
len(data)

43905

In [17]:
data.head(3)


,Merchant Cust ID,Lead_ID,Entity_Type,Node_id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,...,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,solution_type
0,60615636,ebbee93a-4e0b-4e72-a337-31bba8873c8f,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-15 21:46:45.000,2025-04-16 09:29:36.000,NaN,NaN,...,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,60615636,NaN,gJkJJy07871731905616,2025-04-16 09:25:39.000,NaN,NaN
2,1439811339,223ebbf0-85ee-45f2-acc1-d77324831c09,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-16 12:44:47.000,2025-04-16 13:15:34.000,NaN,NaN,...,NaN,NaN,"{""QC_REJECTED"":1,""PANEL_SUCCESS"":1}",NaN,1439811339,NaN,AxdVGE78268205132974,2025-04-16 13:07:09.000,NaN,NaN
6,1312985574,ae574c05-c9ad-42f5-a645-1d674fe4a7a7,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-11 20:52:40.000,2025-04-12 08:55:59.000,NaN,NaN,...,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1312985574,NaN,isceDl72329004783334,2025-04-12 08:48:19.000,NaN,NaN


In [18]:
data['Updated Timestamp']=pd.to_datetime(data['Updated Timestamp'])
data['Created Timestamp']=pd.to_datetime(data['Created Timestamp'])
data['QC Panel Successful TimeStamp']=pd.to_datetime(data['QC Panel Successful TimeStamp'])


In [19]:
user_date = '2024-10-15'  # Example: YYYY-MM-DD format

# Convert the combined string into a datetime object
custom_timestamp = datetime.strptime(user_date, '%Y-%m-%d')

print(custom_timestamp)
print(type(custom_timestamp))

current_day = custom_timestamp.day
current_month = custom_timestamp.month
current_year = custom_timestamp.year

print(current_day)
print(current_month)
print(current_year)


2024-10-15 00:00:00
<class 'datetime.datetime'>
15
10
2024


In [20]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[(data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Last 7 days'

2025-04-21 13:51:54.343113


In [21]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[(data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Yesterday'

In [22]:
data['Ageing']=data['Ageing'].fillna('')

In [23]:
data['Ageing'].value_counts()

               32726
Last 7 days     9970
Yesterday       1209
Name: Ageing, dtype: int64

In [24]:


data.loc[data['Name_Match_Status']==True, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Matched','10k MID created']

data.loc[data['Name_Match_Status']==False, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Not Matched','1Rs MID created']

In [25]:
data['Name_Match_Status']=data['Name_Match_Status'].fillna('')

In [26]:
data['Name_Match_Status'].value_counts()

Name Matched        37870
Name Not Matched     6035
Name: Name_Match_Status, dtype: int64

In [27]:
data['Product_Context'].value_counts()

Series([], Name: Product_Context, dtype: int64)

In [28]:
data['mid_created_date']=pd.to_datetime(data['mid_created_date'],errors='coerce')

In [29]:
data['mid_created_date'].head(10)

0    2025-04-16 09:25:39
2    2025-04-16 13:07:09
6    2025-04-12 08:48:19
13                   NaT
15   2019-11-04 12:32:19
20   2025-04-16 09:30:15
21   2025-04-19 10:24:54
22   2025-04-17 15:06:51
25   2025-04-05 17:01:28
27   2025-04-06 10:17:46
Name: mid_created_date, dtype: datetime64[ns]

In [30]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [31]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day<current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='More Than 7 Days'

2025-04-21 13:51:54.893058


In [32]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='6-7 Days'

In [33]:
current_date = datetime.now() - timedelta(days=5)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='4-5 Days'

In [34]:
current_date = datetime.now() - timedelta(days=3)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='2-3 Days'

In [35]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='Less Than 1 Day'

In [36]:
data['Rejection Ageing'].value_counts()

More Than 7 Days    6120
4-5 Days             699
2-3 Days             696
6-7 Days             670
Less Than 1 Day      312
Name: Rejection Ageing, dtype: int64

In [37]:
data['Rejection Ageing'].value_counts()

More Than 7 Days    6120
4-5 Days             699
2-3 Days             696
6-7 Days             670
Less Than 1 Day      312
Name: Rejection Ageing, dtype: int64

In [38]:
data.loc[~data['Entity_Type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'Entity_Type']='NON_PROPRIETORSHIP'

In [39]:
data['Entity_Type'].value_counts()

INDIVIDUAL            39134
PROPRIETORSHIP         4749
NON_PROPRIETORSHIP       22
Name: Entity_Type, dtype: int64

In [40]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [41]:
data['Lead Status'].value_counts()

Merchant Live                                                       35236
Documents Rejected (Pending at Sales Team)                           8497
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         169
Lead Created/Posted (Pending at Sales team for  Document Upload)        3
Name: Lead Status, dtype: int64

In [42]:
null_rejected_timeStamp=data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]


null_rejected_timeStamp=null_rejected_timeStamp[['Lead_ID']]

print(len(null_rejected_timeStamp))

null_rejected_timeStamp['id']="'" + null_rejected_timeStamp['Lead_ID'] + "'" + ","

null_rejected_timeStamp.head(3)

#null_rejected_timeStamp.to_csv("rejected_null.csv", index=False)


0


,Lead_ID,id


In [43]:
# raise Exception("Stop execution here")

print(len(data))

43905


In [44]:
#rejected_null_value=pd.read_csv('rejected_null_value.csv')

#rejected_null_value['q']=pd.to_datetime(rejected_null_value['q'],format='%d-%m-%Y %H:%M:%S')

#print(rejected_null_value.columns)

In [45]:
#print(rejected_null_value.head(3))
print(len(data))

43905


In [46]:

# Aggregate the right DataFrame by taking the first occurrence per lead_id
#rejected_agg = rejected_null_value.groupby('lead_id').first().reset_index()

# Merge the DataFrames on the single column
#data = data.merge(rejected_agg[['lead_id', 'Status']], left_on='Lead_ID', right_on='lead_id', how='left')

# Fill in values based on conditions
#data.loc[(data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()), 'Last QC Pending Timestamp'] = data['q']


In [47]:
#data=data.merge(rejected_null_value,left_on='Lead_ID',right_on='lead_id',how='left')

#data.loc[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()) ,'Last QC Pending Timestamp']=data['q']

#data=data.drop(columns=['lead_id','q'])

print(data.columns)
print(len(data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]))
print(len(data))

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')
0
43905


In [48]:
data['Final TimeStamp']=pd.to_datetime(data['Final TimeStamp'])
data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'])

In [49]:
# type(data['Final TimeStamp'][0])
#print(type(data['Last QC Pending Timestamp'][0]))
print(len(data))

43905


In [50]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [51]:
data['Last QC Pending Timestamp'].head(10)

0    2025-04-15 21:53:55
2    2025-04-16 13:04:25
6    2025-04-11 21:03:16
13   2025-04-21 20:58:39
15   2025-04-16 18:50:22
20   2025-04-15 22:15:05
21   2025-04-19 09:28:20
22   2025-04-17 15:05:08
25   2025-04-09 19:28:54
27   2025-04-05 20:36:58
Name: Last QC Pending Timestamp, dtype: datetime64[ns]

In [52]:


data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')

# Define the start and end times
#start_time = time(8, 30, 0)
#end_time = time(21, 30, 0)

start_time = time(8, 00, 0)
end_time = time(22, 00, 0)



# Apply the condition to classify as 'Intime' or 'Offtime' based on the time component of 'N'
data['Working TimeSlab'] = data['Last QC Pending Timestamp'].apply(lambda x: 'Offtime' if pd.isna(x) else ('Intime' if start_time <= x.time() <= end_time else 'Offtime'))


data['Working TimeSlab'].head(10)

0      Intime
2      Intime
6      Intime
13     Intime
15     Intime
20    Offtime
21     Intime
22     Intime
25     Intime
27     Intime
Name: Working TimeSlab, dtype: object

In [53]:
data['Working TimeSlab'].value_counts()

Intime     37236
Offtime     6669
Name: Working TimeSlab, dtype: int64

In [54]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [55]:
#out_time_start = pd.to_datetime("21:30").time()
#out_time_end = pd.to_datetime("23:59:59").time()
#morning_time_end = pd.to_datetime("08:30").time()

out_time_start = pd.to_datetime("22:00").time()
out_time_end = pd.to_datetime("23:59:59").time()
morning_time_end = pd.to_datetime("08:00").time()

data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')


# Apply conditions to create column R
def process_timestamp(row):
    timestamp = row['Last QC Pending Timestamp']
    
    # Check if timestamp is NaT
    if pd.isna(timestamp):
        return np.nan
    
    time = timestamp.time()
    
    if row['Working TimeSlab'] == 'Offtime':
        if out_time_start <= time <= out_time_end:
            # Condition 1: Increment date by 1
            return timestamp + pd.DateOffset(days=1)
        elif time < morning_time_end:
            # Condition 2: Return the same timestamp
            return timestamp
    # Condition 3: Return blank for 'intime'
    return np.nan


data['New Off Time'] = data.apply(process_timestamp, axis=1)

In [56]:
# Function to create S column based on R
def generate_s(row):
    if pd.notna(row['New Off Time']):  # If R is not NaT or blank
        # Set time to 08:30 on the same date as R
        return row['New Off Time'].replace(hour=8, minute=00, second=0)
    else:
        # If R is NaT or blank, copy the value from Q
        return row['Last QC Pending Timestamp']
    

data['New Last QC Pending Time'] = data.apply(generate_s, axis=1)

In [57]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [58]:
# Calculate the difference between L and S and format it as HH:mm:ss
def calculate_difference(row):
    if isinstance(row['New Last QC Pending Time'], pd.Timestamp) and isinstance(row['Final TimeStamp'], pd.Timestamp):
        diff = row['Final TimeStamp'] - row['New Last QC Pending Time']
        return diff  # Extract HH:mm:ss from Timedelta
    return np.nan  # Return NaN if either L or S is not a timestamp

# Apply function to create Difference column
data['TAT'] = data.apply(calculate_difference, axis=1)

In [59]:
data['TAT'].head(10)

0    0 days 11:31:38
2    0 days 00:02:31
6    0 days 11:44:58
13   0 days 00:02:02
15   0 days 00:08:01
20   0 days 01:30:08
21   0 days 00:56:24
22   0 days 00:01:33
25   0 days 12:52:02
27   0 days 13:40:44
Name: TAT, dtype: timedelta64[ns]

In [60]:
# Update TAT where it is negative to be the difference between L and N
def adjust_negative_tat(row):
    if pd.notna(row['TAT']) and row['TAT'] < pd.Timedelta(0):  # Check if TAT is negative
        if isinstance(row['Final TimeStamp'], pd.Timestamp) and isinstance(row['Last QC Pending Timestamp'], pd.Timestamp):
            return row['Final TimeStamp'] - row['Last QC Pending Timestamp']  # Calculate L - N if TAT is negative
    return row['TAT']  # Keep original TAT if not negative

# Apply function to adjust negative TAT values
data['TAT'] = data.apply(adjust_negative_tat, axis=1)

In [61]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

       Final TimeStamp Last QC Pending Timestamp             TAT
0  2025-04-16 09:25:33       2025-04-15 21:53:55 0 days 11:31:38
2  2025-04-16 13:06:56       2025-04-16 13:04:25 0 days 00:02:31
6  2025-04-12 08:48:14       2025-04-11 21:03:16 0 days 11:44:58
13 2025-04-21 21:00:41       2025-04-21 20:58:39 0 days 00:02:02
15 2025-04-16 18:58:23       2025-04-16 18:50:22 0 days 00:08:01
20 2025-04-16 09:30:08       2025-04-15 22:15:05 0 days 01:30:08
21 2025-04-19 10:24:44       2025-04-19 09:28:20 0 days 00:56:24
22 2025-04-17 15:06:41       2025-04-17 15:05:08 0 days 00:01:33
25 2025-04-10 08:20:56       2025-04-09 19:28:54 0 days 12:52:02
27 2025-04-06 10:17:42       2025-04-05 20:36:58 0 days 13:40:44
43905


In [62]:
#len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())]

,Merchant Cust ID,Lead_ID,Entity_Type,Node_id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,...,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,solution_type


In [63]:
data = data[~(((data['Lead Status'] == 'Merchant Live') | (data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)')) & (data['TAT'].isnull()))]


len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

0

In [64]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'leads QCed in more than 01 hr'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)



In [65]:
data['Merchant QC TAT slab'].value_counts()

leads QCed in more than 01 hr           14191
Leads QCed in less than 02 min          13844
Leads QCed in between 02 - 05 min        7730
Leads QCed in between 05 - 10 min        3232
Leads QCed in between 30 min - 01 hr     2225
Leads QCed in between 15 - 30 min        1527
Leads QCed in between 10 - 15 min         984
Name: Merchant QC TAT slab, dtype: int64

In [66]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [67]:
data['solution_type_level_3'].unique()

array([nan, 'diy_upgrade'], dtype=object)

In [68]:
data.loc[(data['Entity_Type']=='INDIVIDUAL') & (data['solution_type_level_3'].isna()),'solution_type']='DIY_MCO'
data.loc[(data['Entity_Type']=='INDIVIDUAL') & (~data['solution_type_level_3'].isna()),'solution_type']='DIY_ReKYC'
data.loc[~(data['Entity_Type']=='INDIVIDUAL'),'solution_type']='DIY_Upgrade'

In [69]:
data['solution_type'].unique()

array(['DIY_MCO', 'DIY_Upgrade', 'DIY_ReKYC'], dtype=object)

In [70]:
data.to_csv("diy_mco_data_27th April'25.csv", index=False)

In [71]:
hzdjz

NameError: name 'hzdjz' is not defined

In [ ]:
#data.to_csv("all_flow_data_28th Oct'24.csv", index=False)

In [ ]:
sas

In [ ]:


# Convert to datetime
data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'])
data['Final TimeStamp'] = pd.to_datetime(data['Final TimeStamp'])

# Define active hours (08:30 AM - 09:30 PM)
active_start_time = pd.to_timedelta('08:30:00')
active_end_time = pd.to_timedelta('21:30:00')

# Function to calculate active duration for each row
def calculate_active_duration(row):
    start = row['Last QC Pending Timestamp']
    end = row['Final TimeStamp']
    
    # Check if either start or end is NaT
    if pd.isna(start) or pd.isna(end):
        return "00:00:00"  # Return 0 hours if either is NaT
    
    # Initialize total active duration
    total_active_duration = pd.Timedelta(0)
    
    # Iterate over each date from start to end
    for single_date in pd.date_range(start=start.date(), end=end.date()):
        active_start = pd.Timestamp(single_date) + active_start_time
        active_end = pd.Timestamp(single_date) + active_end_time
        
        # Determine the effective active period for this day
        effective_start = max(start, active_start)
        effective_end = min(end, active_end)

        # If there's a valid active period, add its duration
        if effective_start < effective_end:
            total_active_duration += effective_end - effective_start

    # Convert total active duration to hours
    total_hours = total_active_duration.total_seconds() / 3600
    
    # Format the hours to HH:MM:SS
    hours = int(total_hours)
    minutes = int((total_hours - hours) * 60)
    seconds = int(((total_hours - hours) * 60 - minutes) * 60)
    
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Apply the function to each row
data['TAT'] = data.apply(calculate_active_duration, axis=1)

# Display the result
print(data[['Last QC Pending Timestamp', 'Final TimeStamp', 'TAT']])
print(len(data))


In [ ]:
# data['TAT']=data['Final TimeStamp']-data['Last QC Pending Timestamp']
# data['TAT']=data['TAT'].apply(lambda x: str(x).split()[-1] if pd.notna(x) else 'NaT')
print(data.columns)
print(len(data))

In [ ]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

In [ ]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

In [ ]:
data["TAT"]=pd.to_timedelta(data['TAT'])



print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))

In [ ]:
type(data['TAT'][0])

In [ ]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 16:
        return 'Leads QCed in less than 15 min'
    elif 16 <= total_minutes < 36:
        return 'Leads QCed in between 15-30 min'
    elif 36 <= total_minutes < 66:
        return 'Leads QCed in between 30 min -1 hr'
    elif 66 <= total_minutes < 126:
        return 'Leads QCed in between 1 - 2 hr'
    elif 126 <= total_minutes < 186:
        return 'Leads QCed in between 2-3 hr'
    elif 186 <= total_minutes < 246:
        return 'Leads QCed in between 3-4 hr'
    else:
        return 'leads QCed in more than 4 hrs'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)


    



In [ ]:
print(data['Merchant QC TAT slab'].value_counts())
print(len(data))

In [ ]:
data['TAT'].head(10)

In [ ]:

# Function to convert timedelta to HH:MM:SS format with cumulative hours
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)  # Convert total seconds to hours
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'  # Format as HH:MM:SS

# Apply the function to convert timedelta to HH:MM:SS
data['TAT'] = data['TAT'].apply(format_timedelta)

In [ ]:
data.loc[(data['Lead Status']!='Merchant Live') & (data['Lead Status']!='Documents Rejected (Pending at Sales Team)'),'TAT']=''

data['TAT'].head(10)

In [ ]:
data.columns

In [ ]:
data=data.drop(columns=['Working TimeSlab','New Off Time','New Last QC Pending Time','Node id'])
data.columns

In [ ]:
col=data.pop('TAT')

t=data.columns.get_loc('Merchant QC TAT slab')

data.insert(t+1,'TAT',col)

data.columns

In [ ]:
data['solution_type_level_3'].unique()
len(data)

In [ ]:
data_acup_kyc=data[data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]
data_mco=data[~data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]

In [ ]:
#raise Exception("Stop execution here")

In [ ]:
with pd.ExcelWriter("all_flow_data_21st Oct'24.xlsx") as writer:
    # Write the original dataframe to the first sheet
    data.to_excel(writer, sheet_name='All Data', index=False)
    
    # Write data filtered by Category 'A' and 'C' to the second sheet
    data_acup_kyc.to_excel(writer, sheet_name=' KYC & ACC UPGD', index=False)
    
    # Write data filtered by Category 'B' and 'D' to the third sheet
    data_mco.to_excel(writer, sheet_name='MCO', index=False)

In [ ]:
data.to_csv("all_flow_data_19th Oct'24.csv", index=False)
